# Voxel pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher
from VoxelGrid import VoxelGrid

%matplotlib ipympl

### Load calibrations and other data

In [2]:
imageProcessor = ImageProcessor()
imageProcessor.verbose = True
imageProcessor.loadMonoCalibrationResults()
imageProcessor.loadStereoCalibration()
imageProcessor.loadCameraProperties()
imageProcessor.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/cameraProperties.json
Loaded camera properties
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [3]:
stereoMatcher = StereoMatcher(imageProcessor=imageProcessor, \
                matcher="SGBM", vertical=True, createRightMatcher=False)

imageProcessor.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [4]:
path = "testImages/voxelTestImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-0


### Select image pair and display

In [5]:
imageNumber = 0

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

plt.figure()
plt.imshow(cv2.cvtColor(np.hstack([imageL, imageL]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Convert to grayscale and undistort

In [6]:
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

### View undistorted image

In [7]:
fig = plt.figure()
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig = plt.figure()
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig = plt.figure(figsize=(6,15))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [10]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 9.0
maxDisparity: 41.0


### View disparity map

In [11]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [12]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
41.0
(640, 360)
481.33795
2192.7617


### View depth map

In [13]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Use to save images
# _=cv2.imwrite("depth.png", cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

### Compute point cloud

In [15]:
voxelGrid = VoxelGrid(stereoMatcher, imageProcessor)
voxelGrid.verbose = True

In [16]:
voxelGrid.generatePointCloud()

Points in unfiltered pointcloud: 11520; completed in 0.00203 sec


Filtering extreme points

In [17]:
voxelGrid.filterPointCloud()

Points in filtered pointcloud: 9245; completed in 0.00076 sec


Rotate point cloud to have y forward

In [18]:
voxelGrid.redefinePointCloudCoordinate()

### View point cloud

In [19]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.pointCloud[:,0], voxelGrid.pointCloud[:,1], \
    voxelGrid.pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at x=0 and looks to positive x
ax.set_xlim(0,2000)
ax.set_ylim(-1000,1000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [20]:
voxelGrid.resetVoxelGrid()
voxelGrid.voxelizePointCloud()

Voxel grid reset
Voxels in grid: 59; completed in 0.00913 sec; 70 iterations


### View voxelized point cloud

In [21]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.voxelGrid[:,0], \
        voxelGrid.voxelGrid[:,1], \
        voxelGrid.voxelGrid[:,2])

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at x=0 and looks to positive x
ax.set_xlim(0,2000)
ax.set_ylim(-1000,1000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Refine voxel grid with newer data

Interchanging data from calibration since calibration was 
done vertically

The x y notations are in image coordinates

In [22]:
# Horizontal field of view (degrees)
fovH = ((imageProcessor.fovYL+imageProcessor.fovYR)/4)*np.pi/180
fovH -= fovH/8
print("Horizontal:", fovH*180/np.pi)

# Vertical field of view (degrees)
fovV = ((imageProcessor.fovXL+imageProcessor.fovXR)/4)*np.pi/180
fovV -= fovV/8
print("Vertical:", fovV*180/np.pi)

Horizontal: 24.915932084055157
Vertical: 14.867033874651561


### Checking for voxels in range

The refinement is done by simply replacing the older voxels in view of the camera in the base gridwith the new voxels.

Here the voxels that may potentially be affected are found.

In [23]:
# Do this after compensations for atleast camera translation have been made
# Performed on the base voxel grid
# Distances to every voxel in base grid from camera

# Offset camera position with distance from robot center and distance of robot 
# from origin
# The top or left camera is taken to represent the whole camera grid
cameraPosition = np.zeros([3])
distanceToVoxels = np.linalg.norm(voxelGrid.voxelGrid-cameraPosition, axis=1)
print(distanceToVoxels.shape)

# Distance within which points may be modified
distanceToCheck = 1500

# These are the points to check
# Performed on the base voxel grid
voxelsWithinRange = voxelGrid.voxelGrid[distanceToVoxels<=distanceToCheck]
print(voxelGrid.voxelGrid.shape)
print(voxelsWithinRange.shape)

# Checking bounds on the coordinate axes
# Performed on the base voxel grid
voxelCheckBound = [[voxelsWithinRange[:,0].min(), voxelsWithinRange[:,1].min(), \
                        voxelsWithinRange[:,2].min()], \
                    [voxelsWithinRange[:,0].max(), voxelsWithinRange[:,1].max(), \
                        voxelsWithinRange[:,2].max()]]
print(voxelCheckBound) # min, max

(59,)
(59, 3)
(59, 3)
[[550, -550, -250], [1150, 250, 250]]


### Angle ranges of rotated new voxel grid (Exploration only)

Could potentially be found from the rotation matrix and be offset 
by the camera fovs.

Computing yaws on the new voxel grid; xy plane. A line along x axis has 0 degrees of yaw.

Normally this would be the results of the latest iteration after 
they are rotated and translated.

Taking x forward

In [30]:
newVoxelGrid = voxelGrid.voxelGrid
yawToNewVoxels = np.arctan2(newVoxelGrid[:,1], newVoxelGrid[:,0]) # y, x
print(yawToNewVoxels.shape)

yawCheckBound = np.array([yawToNewVoxels.max(), yawToNewVoxels.min()]) # left, right
print(yawCheckBound*180/np.pi)
print((yawCheckBound[0]-yawCheckBound[1])*180/np.pi)

#print(yawToNewVoxels*180/np.pi)

(59,)
[ 24.443953 -25.559963]
50.0039173665009


Computing pitches on the new voxel grid; xz plane.

Taking x forward

In [31]:
pitchToNewVoxels = np.arctan2(newVoxelGrid[:,2], newVoxelGrid[:,0])
print(pitchToNewVoxels.shape)

                                                                # up, down
pitchCheckBound = np.array([pitchToNewVoxels.max(), pitchToNewVoxels.min()]) 
print(pitchCheckBound*180/np.pi)
print((pitchCheckBound[0]-pitchCheckBound[1])*180/np.pi)

#print(pitchToNewVoxels*180/np.pi)

(59,)
[ 14.743562 -16.38954 ]
31.13310189737449


Computing rolls on the new voxel grid; yz plane.

Taking x forward

In [32]:
rollToNewVoxels = np.arctan2(newVoxelGrid[:,2], newVoxelGrid[:,1])
print(rollToNewVoxels.shape)

rollCheckBound = np.array([rollToNewVoxels.max(), rollToNewVoxels.min()])
print(rollCheckBound*180/np.pi)
print((rollCheckBound[0]-rollCheckBound[1])*180/np.pi)

#print(rollToNewVoxels*180/np.pi)

(59,)
[ 173.65982 -173.65982]
347.3196272200006


From the results above, checking for pitch and roll will return 
inconclusive results unless the rotation is in an exact side profile of the
grid.

This can be mitigated by doing the check before rotating or translating, but that would require the base grid to be rotated as a whole, refined, and then re-rotated back.

To save on computation, the yaw range for the new voxel grid may simply be taken from camera properties already found during calibration. 

Then, to check where the camera is facing, a vector of unit distance from the camera facing along its axis could be rotated and translated along with the camera. 

Its yaw with respect to the camera center can then be calculated, and then offset by half its horizontal fov on both sides, to find the yaw limits within which the base voxels may be modified.

### Finding yaw of camera and voxels within range
Finding yaw of camera

In [40]:
# Camera vector before rotation, z forward
cameraDirectionVector = np.array([0,0,100])
print(cameraDirectionVector)

# Camera vector after rotation (do not translate)
# Also perform rotations to align with the robot frame, not done here
cameraDirectionVector = np.dot(\
            cameraDirectionVector, voxelGrid.redefineRotationMatrix)
print(cameraDirectionVector)

# Finding yaw of the camera vector (again, assuming no translation)
cameraYaw = np.arctan2(cameraDirectionVector[1], cameraDirectionVector[0])
print(cameraYaw*180/np.pi)

cameraYawRange = np.array([cameraYaw+fovH, cameraYaw-fovH])
print(cameraYawRange)
print(cameraYawRange*180/np.pi)

# Wrapping around values at -180, 180 degrees
for n in range(len(cameraYawRange)):
    if cameraYawRange[n]>np.pi:
        cameraYawRange[n] -= 2*np.pi
    if cameraYawRange[n]<=-np.pi:
        cameraYawRange[n] += 2*np.pi

cameraYawRange = np.sort(cameraYawRange)[::-1]

print(cameraYawRange)
print(cameraYawRange*180/np.pi)

[  0   0 100]
[100   0   0]
0.0
[ 0.43486505 -0.43486505]
[ 24.91593208 -24.91593208]
[ 0.43486505 -0.43486505]
[ 24.91593208 -24.91593208]


Finding yaw of voxels within range

In [46]:
# Shifting the voxels around the camera to origin so that yaw to each voxel 
# can be found
translatedVoxelsWithinRange = voxelsWithinRange-cameraPosition
print(translatedVoxelsWithinRange.shape)

yawToTranslatedVoxels = np.arctan2(translatedVoxelsWithinRange[:,1], \
                                translatedVoxelsWithinRange[:,0])
print(yawToTranslatedVoxels.shape)
#print(yawToTranslatedVoxels*180/np.pi)

(59, 3)
(59,)


Finding voxels that need to be removed

In [45]:
if cameraYawRange[0]>np.pi/2 and cameraYawRange[1]<-np.pi/2:
    voxelsToRemove = voxelsWithinRange[np.logical_and(\
        yawToTranslatedVoxels[:]>cameraYawRange[0], \
        yawToTranslatedVoxels[:]<cameraYawRange[1]
        )]
else:
    voxelsToRemove = voxelsWithinRange[np.logical_and(\
        yawToTranslatedVoxels[:]<cameraYawRange[0], \
        yawToTranslatedVoxels[:]>cameraYawRange[1]
        )]

print(voxelsToRemove.shape)

(57, 3)
